[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/03_Audit_Trail_and_Report.ipynb)


In [ ]:
import sys
from pathlib import Path

repo_root = Path().resolve()
src_path = repo_root / 'src'
if src_path.exists():
    sys.path.insert(0, str(src_path))
else:
    print('If running in Colab, install with: pip install iints-sdk-python35')


# Audit Trail and Clinical Report

This notebook shows how to export the audit log and generate a PDF report.


In [ ]:
import json
from pathlib import Path

import iints
from iints.core.algorithms.pid_controller import PIDController

presets = json.loads(Path("src/iints/presets/presets.json").read_text())
preset = next(p for p in presets if p["name"] == "baseline_t1d")

outputs = iints.run_simulation(
    algorithm=PIDController,
    scenario=preset["scenario"],
    patient_config=preset["patient_config"],
    duration_minutes=240,
    time_step=preset["time_step_minutes"],
    seed=11,
    output_dir="results/audit_report",
    export_audit=True,
    generate_report=True,
)

outputs["audit"], outputs["report_pdf"]


In [ ]:
import json

summary_path = Path(outputs["audit"]["summary"])
json.loads(summary_path.read_text())
